In [1]:
following figures are fromimport numpy as np
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [2]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [3]:
for i in range(len(y_train_valid)):
    if(y_train_valid[i] == 769):
        y_train_valid[i] = 0
    if(y_train_valid[i] == 770):
        y_train_valid[i] = 1
    if(y_train_valid[i] == 771):
        y_train_valid[i] = 2
    if(y_train_valid[i] == 772):
        y_train_valid[i] = 3

In [4]:
for i in range(len(y_test)):
    if(y_test[i] == 769):
        y_test[i] = 0
    if(y_test[i] == 770):
        y_test[i] = 1
    if(y_test[i] == 771):
        y_test[i] = 2
    if(y_test[i] == 772):
        y_test[i] = 3

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
datatype = torch.float
torch.cuda.device(device)
print(device)

cuda:0


In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Spatial Convolution
        self.conv1 = nn.Conv2d(1, 40, (22, 1), stride = (1, 1))
        nn.init.xavier_uniform_(self.conv1.weight)
        # Batch Normalization
        self.batch1 = nn.BatchNorm2d(40)
        # ELU
        self.elu1 = nn.ELU()
        # Temporal Convolution
        self.conv2 = nn.Conv2d(40, 40, (1, 40), stride = (1, 2))
        nn.init.xavier_uniform_(self.conv2.weight)
        # Batch Normalization
        self.batch2 = nn.BatchNorm2d(40)
        # ELU
        self.elu2 = nn.ELU()
        # Avg Pooling
        self.pool1 = nn.AvgPool2d((1, 75), stride = (1, 15))
        # Dropout
        self.dropout = nn.Dropout(p = 0.3)
        # Fully Connected
        self.fc1 = nn.Linear(40 * 28, 4)
        # Softmax
        self.softmax = nn.Softmax(1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.batch1(x)
        x = self.elu1(x)
        x = self.conv2(x)
        x = self.batch2(x)
        x = self.elu2(x)
        x = self.pool1(x)
        x = self.dropout(x)
        x = x.reshape(x.shape[0], x.shape[1] * x.shape[2] * x.shape[3])
        x = self.fc1(x)
        x = self.softmax(x)
        return x

In [8]:
cnn = CNN().to(device)

In [9]:
lr = 0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn.parameters(), lr=lr)

In [10]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 4)

In [11]:
batch_size = 500
num_iters = 1
num_epoch = 1000
epoch_loss = [0.0 for i in range(num_epoch)]
train_accuracy = [0.0 for i in range(num_epoch)]
validation_accuracy = [0.0 for i in range(num_epoch)]
for epoch in range(num_epoch):
    counter = 0
    for train_index, valid_index in kf.split(X_train_valid):
        X_train_cur = torch.tensor(X_train_valid[train_index].reshape(X_train_valid[train_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        X_valid_cur = torch.tensor(X_train_valid[valid_index].reshape(X_train_valid[valid_index].shape[0], -1, 22, 1000), dtype=torch.float).to(device)
        y_train_cur = torch.tensor(y_train_valid[train_index], dtype=torch.long).to(device)
        y_valid_cur = torch.tensor(y_train_valid[valid_index], dtype=torch.long).to(device)
        for it in np.arange(num_iters):
            index = np.random.choice(X_train_cur.shape[0], batch_size)
            X_batch = X_train_cur[index, :, :, :]
            y_batch = y_train_cur[index]
            optimizer.zero_grad()
            outputs = cnn(X_batch)

            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss[epoch] += loss

            counter += 1
            
    # Validation Accuracy for final K-fold
        output_val = cnn(X_valid_cur)
        prediction_val = torch.argmax(output_val, axis = 1)
        accuracy_val = float(prediction_val.eq(y_valid_cur).sum()) / float(y_valid_cur.shape[0])
        print('Current Epoch: {}, Validation Accuracy: {}'.format(epoch, accuracy_val))

    # Training Accuracy for final K-fold
        output_train = cnn(X_train_cur)
        prediction_train = torch.argmax(output_train, axis = 1)
        accuracy_train = float(prediction_train.eq(y_train_cur).sum()) / float(y_train_cur.shape[0])
    
        train_accuracy[epoch] += accuracy_train
        validation_accuracy[epoch] += accuracy_val
    epoch_loss[epoch] /= counter 
    print('Current Epoch: {}, Epoch Loss: {}'.format(epoch, epoch_loss[epoch]))

Current Epoch: 0, Validation Accuracy: 0.34215500945179583
Current Epoch: 0, Validation Accuracy: 0.3383742911153119
Current Epoch: 0, Validation Accuracy: 0.43100189035916825
Current Epoch: 0, Validation Accuracy: 0.3977272727272727
Current Epoch: 0, Epoch Loss: 1.3518328666687012
Current Epoch: 1, Validation Accuracy: 0.3988657844990548
Current Epoch: 1, Validation Accuracy: 0.4499054820415879
Current Epoch: 1, Validation Accuracy: 0.5519848771266541
Current Epoch: 1, Validation Accuracy: 0.48295454545454547
Current Epoch: 1, Epoch Loss: 1.2715442180633545
Current Epoch: 2, Validation Accuracy: 0.45935727788279773
Current Epoch: 2, Validation Accuracy: 0.5217391304347826
Current Epoch: 2, Validation Accuracy: 0.5822306238185255
Current Epoch: 2, Validation Accuracy: 0.5511363636363636
Current Epoch: 2, Epoch Loss: 1.2146631479263306
Current Epoch: 3, Validation Accuracy: 0.500945179584121
Current Epoch: 3, Validation Accuracy: 0.5671077504725898
Current Epoch: 3, Validation Accuracy:

Current Epoch: 29, Validation Accuracy: 0.667296786389414
Current Epoch: 29, Validation Accuracy: 0.6918714555765595
Current Epoch: 29, Validation Accuracy: 0.7712665406427222
Current Epoch: 29, Validation Accuracy: 0.7367424242424242
Current Epoch: 29, Epoch Loss: 1.0211846828460693
Current Epoch: 30, Validation Accuracy: 0.6880907372400756
Current Epoch: 30, Validation Accuracy: 0.7069943289224953
Current Epoch: 30, Validation Accuracy: 0.7637051039697542
Current Epoch: 30, Validation Accuracy: 0.75
Current Epoch: 30, Epoch Loss: 1.0162349939346313
Current Epoch: 31, Validation Accuracy: 0.6786389413988658
Current Epoch: 31, Validation Accuracy: 0.7410207939508506
Current Epoch: 31, Validation Accuracy: 0.7637051039697542
Current Epoch: 31, Validation Accuracy: 0.7537878787878788
Current Epoch: 31, Epoch Loss: 0.9972764849662781
Current Epoch: 32, Validation Accuracy: 0.6843100189035917
Current Epoch: 32, Validation Accuracy: 0.6899810964083176
Current Epoch: 32, Validation Accuracy:

Current Epoch: 58, Validation Accuracy: 0.7655954631379962
Current Epoch: 58, Validation Accuracy: 0.7523629489603024
Current Epoch: 58, Validation Accuracy: 0.8166351606805293
Current Epoch: 58, Validation Accuracy: 0.8011363636363636
Current Epoch: 58, Epoch Loss: 0.9665623307228088
Current Epoch: 59, Validation Accuracy: 0.7731568998109641
Current Epoch: 59, Validation Accuracy: 0.7580340264650284
Current Epoch: 59, Validation Accuracy: 0.8109640831758034
Current Epoch: 59, Validation Accuracy: 0.8125
Current Epoch: 59, Epoch Loss: 0.9685226678848267
Current Epoch: 60, Validation Accuracy: 0.7637051039697542
Current Epoch: 60, Validation Accuracy: 0.776937618147448
Current Epoch: 60, Validation Accuracy: 0.8090737240075614
Current Epoch: 60, Validation Accuracy: 0.8011363636363636
Current Epoch: 60, Epoch Loss: 0.9551810026168823
Current Epoch: 61, Validation Accuracy: 0.7485822306238186
Current Epoch: 61, Validation Accuracy: 0.7674858223062382
Current Epoch: 61, Validation Accurac

Current Epoch: 87, Validation Accuracy: 0.8109640831758034
Current Epoch: 87, Validation Accuracy: 0.8185255198487713
Current Epoch: 87, Validation Accuracy: 0.8298676748582231
Current Epoch: 87, Validation Accuracy: 0.8276515151515151
Current Epoch: 87, Epoch Loss: 0.9377816915512085
Current Epoch: 88, Validation Accuracy: 0.7977315689981096
Current Epoch: 88, Validation Accuracy: 0.8034026465028355
Current Epoch: 88, Validation Accuracy: 0.8676748582230623
Current Epoch: 88, Validation Accuracy: 0.821969696969697
Current Epoch: 88, Epoch Loss: 0.9459637999534607
Current Epoch: 89, Validation Accuracy: 0.8034026465028355
Current Epoch: 89, Validation Accuracy: 0.775047258979206
Current Epoch: 89, Validation Accuracy: 0.8468809073724007
Current Epoch: 89, Validation Accuracy: 0.8371212121212122
Current Epoch: 89, Epoch Loss: 0.9523084759712219
Current Epoch: 90, Validation Accuracy: 0.8034026465028355
Current Epoch: 90, Validation Accuracy: 0.782608695652174
Current Epoch: 90, Validati

Current Epoch: 115, Validation Accuracy: 0.8465909090909091
Current Epoch: 115, Epoch Loss: 0.9151421785354614
Current Epoch: 116, Validation Accuracy: 0.8034026465028355
Current Epoch: 116, Validation Accuracy: 0.8204158790170132
Current Epoch: 116, Validation Accuracy: 0.8695652173913043
Current Epoch: 116, Validation Accuracy: 0.8636363636363636
Current Epoch: 116, Epoch Loss: 0.8979775905609131
Current Epoch: 117, Validation Accuracy: 0.8393194706994329
Current Epoch: 117, Validation Accuracy: 0.831758034026465
Current Epoch: 117, Validation Accuracy: 0.8771266540642723
Current Epoch: 117, Validation Accuracy: 0.8560606060606061
Current Epoch: 117, Epoch Loss: 0.9060438871383667
Current Epoch: 118, Validation Accuracy: 0.8355387523629489
Current Epoch: 118, Validation Accuracy: 0.8279773156899811
Current Epoch: 118, Validation Accuracy: 0.8733459357277883
Current Epoch: 118, Validation Accuracy: 0.8465909090909091
Current Epoch: 118, Epoch Loss: 0.9012710452079773
Current Epoch: 11

...

Current Epoch: 881, Validation Accuracy: 0.9697542533081286
Current Epoch: 881, Validation Accuracy: 0.9734848484848485
Current Epoch: 881, Epoch Loss: 0.7822204828262329
Current Epoch: 882, Validation Accuracy: 0.9659735349716446
Current Epoch: 882, Validation Accuracy: 0.9621928166351607
Current Epoch: 882, Validation Accuracy: 0.9792060491493384
Current Epoch: 882, Validation Accuracy: 0.9772727272727273
Current Epoch: 882, Epoch Loss: 0.7753994464874268
Current Epoch: 883, Validation Accuracy: 0.9621928166351607
Current Epoch: 883, Validation Accuracy: 0.9603024574669187
Current Epoch: 883, Validation Accuracy: 0.9792060491493384
Current Epoch: 883, Validation Accuracy: 0.9753787878787878
Current Epoch: 883, Epoch Loss: 0.7724784016609192
Current Epoch: 884, Validation Accuracy: 0.9659735349716446
Current Epoch: 884, Validation Accuracy: 0.9640831758034026
Current Epoch: 884, Validation Accuracy: 0.9735349716446124
Current Epoch: 884, Validation Accuracy: 0.9678030303030303
Current

Current Epoch: 909, Validation Accuracy: 0.9772727272727273
Current Epoch: 909, Epoch Loss: 0.7761629819869995
Current Epoch: 910, Validation Accuracy: 0.9659735349716446
Current Epoch: 910, Validation Accuracy: 0.9546313799621928
Current Epoch: 910, Validation Accuracy: 0.9773156899810964
Current Epoch: 910, Validation Accuracy: 0.9791666666666666
Current Epoch: 910, Epoch Loss: 0.78154456615448
Current Epoch: 911, Validation Accuracy: 0.9640831758034026
Current Epoch: 911, Validation Accuracy: 0.9584120982986768
Current Epoch: 911, Validation Accuracy: 0.9792060491493384
Current Epoch: 911, Validation Accuracy: 0.9640151515151515
Current Epoch: 911, Epoch Loss: 0.7785452604293823
Current Epoch: 912, Validation Accuracy: 0.9621928166351607
Current Epoch: 912, Validation Accuracy: 0.9640831758034026
Current Epoch: 912, Validation Accuracy: 0.9773156899810964
Current Epoch: 912, Validation Accuracy: 0.9696969696969697
Current Epoch: 912, Epoch Loss: 0.7763552665710449
Current Epoch: 913

Current Epoch: 938, Validation Accuracy: 0.9640831758034026
Current Epoch: 938, Validation Accuracy: 0.9659735349716446
Current Epoch: 938, Validation Accuracy: 0.9810964083175804
Current Epoch: 938, Validation Accuracy: 0.9640151515151515
Current Epoch: 938, Epoch Loss: 0.7738540172576904
Current Epoch: 939, Validation Accuracy: 0.9640831758034026
Current Epoch: 939, Validation Accuracy: 0.9565217391304348
Current Epoch: 939, Validation Accuracy: 0.9810964083175804
Current Epoch: 939, Validation Accuracy: 0.9564393939393939
Current Epoch: 939, Epoch Loss: 0.7750587463378906
Current Epoch: 940, Validation Accuracy: 0.9640831758034026
Current Epoch: 940, Validation Accuracy: 0.9603024574669187
Current Epoch: 940, Validation Accuracy: 0.9810964083175804
Current Epoch: 940, Validation Accuracy: 0.9659090909090909
Current Epoch: 940, Epoch Loss: 0.78163081407547
Current Epoch: 941, Validation Accuracy: 0.9697542533081286
Current Epoch: 941, Validation Accuracy: 0.9640831758034026
Current E

Current Epoch: 966, Validation Accuracy: 0.9640831758034026
Current Epoch: 966, Validation Accuracy: 0.9754253308128544
Current Epoch: 966, Validation Accuracy: 0.9791666666666666
Current Epoch: 966, Epoch Loss: 0.7760940790176392
Current Epoch: 967, Validation Accuracy: 0.9659735349716446
Current Epoch: 967, Validation Accuracy: 0.9659735349716446
Current Epoch: 967, Validation Accuracy: 0.9792060491493384
Current Epoch: 967, Validation Accuracy: 0.9810606060606061
Current Epoch: 967, Epoch Loss: 0.7779574394226074
Current Epoch: 968, Validation Accuracy: 0.9697542533081286
Current Epoch: 968, Validation Accuracy: 0.9603024574669187
Current Epoch: 968, Validation Accuracy: 0.9754253308128544
Current Epoch: 968, Validation Accuracy: 0.9772727272727273
Current Epoch: 968, Epoch Loss: 0.7745531797409058
Current Epoch: 969, Validation Accuracy: 0.9659735349716446
Current Epoch: 969, Validation Accuracy: 0.9603024574669187
Current Epoch: 969, Validation Accuracy: 0.9810964083175804
Current

Current Epoch: 994, Validation Accuracy: 0.9792060491493384
Current Epoch: 994, Validation Accuracy: 0.9772727272727273
Current Epoch: 994, Epoch Loss: 0.7769350409507751
Current Epoch: 995, Validation Accuracy: 0.9678638941398866
Current Epoch: 995, Validation Accuracy: 0.9640831758034026
Current Epoch: 995, Validation Accuracy: 0.9829867674858223
Current Epoch: 995, Validation Accuracy: 0.9753787878787878
Current Epoch: 995, Epoch Loss: 0.7796212434768677
Current Epoch: 996, Validation Accuracy: 0.9678638941398866
Current Epoch: 996, Validation Accuracy: 0.9546313799621928
Current Epoch: 996, Validation Accuracy: 0.9735349716446124
Current Epoch: 996, Validation Accuracy: 0.9772727272727273
Current Epoch: 996, Epoch Loss: 0.7737026214599609
Current Epoch: 997, Validation Accuracy: 0.9659735349716446
Current Epoch: 997, Validation Accuracy: 0.9584120982986768
Current Epoch: 997, Validation Accuracy: 0.9792060491493384
Current Epoch: 997, Validation Accuracy: 0.9583333333333334
Current

In [12]:
X_test = torch.tensor(X_test.reshape(X_test.shape[0], -1, 22, 1000), dtype= torch.float).to(device)
y_test = torch.tensor(y_test, dtype = torch.long).to(device)
output_test = cnn(X_test)
prediction_test = torch.argmax(output_test, axis = 1)
accuracy_test = float(prediction_test.eq(y_test).sum()) / float(y_test.shape[0])
print('Current Epoch: {}, Test Accuracy: {}'.format(epoch, accuracy_test))

Current Epoch: 999, Test Accuracy: 0.690744920993228


In [13]:
torch.save(cnn, 'model_different_order_filter.pt')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [14]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(range(num_epoch), epoch_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('CNN with Different Filtering Order')
fig.savefig('CNN_Different_Filter_Order.png', dpi=fig.dpi)

In [15]:
epoch_loss = [i.cpu() for i in epoch_loss]
epoch_loss = [i.detach().numpy() for i in epoch_loss]
np.save('epoch_loss_CNN_EEG_Different_Filter_order.npy', epoch_loss)

In [23]:
train_accuracy = [i / 4.0 for i in train_accuracy]
np.save('train_acc_CNN_EEG_Different_Filter_order.npy', train_accuracy)

In [24]:
validation_accuracy = [i / 4.0 for i in validation_accuracy]
np.save('val_acc_CNN_EEG_Different_Filter_order.npy', validation_accuracy)

TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [22]:
[i / 4.0 for i in validation_accuracy]

[0.37731461591338716,
 0.47092767228046056,
 0.5286158489431174,
 0.5664382482671708,
 0.5801594632525635,
 0.6132595448817094,
 0.6274291831357048,
 0.623184825571404,
 0.6279071432663115,
 0.6416113521796414,
 0.6510765738672166,
 0.6671553674743657,
 0.6600566749727903,
 0.6732981397147276,
 0.6628984791201238,
 0.6704742366958814,
 0.6789719023887266,
 0.692206206679269,
 0.6974172251818754,
 0.6969365798819958,
 0.7021395428767829,
 0.704977766798419,
 0.6983668800481182,
 0.7049741865727215,
 0.7177475368047201,
 0.7026192931202384,
 0.7125374133585382,
 0.7168059374462966,
 0.7196361058601134,
 0.7167943017127799,
 0.7271975425330812,
 0.7342881795268374,
 0.7229433393481125,
 0.7305155166981727,
 0.7338254353554448,
 0.7413895571976857,
 0.7380867989918084,
 0.7366511284871399,
 0.7395000930858682,
 0.7517865326230166,
 0.7475251689866529,
 0.7498890130033797,
 0.7579185641862864,
 0.762661468178954,
 0.7645563026293178,
 0.7565070602050753,
 0.7654970069313168,
 0.781094260182